In [3]:
import cv2
import numpy as np
import joblib

frameWidth = 240
frameHeight = 240
vc = cv2.VideoCapture(0)
vc.set(3, frameWidth)
vc.set(4, frameHeight)
classifier = joblib.load('Uno_Classifier.sav')

def empty(a):
    pass

cv2.namedWindow('Parameters')
cv2.resizeWindow('Parameters', 240,180)
cv2.createTrackbar('Threshold1','Parameters', 70, 255, empty)
cv2.createTrackbar('Threshold2','Parameters', 190, 255, empty)
cv2.createTrackbar('Area_min','Parameters', 1500, 10000, empty)
cv2.createTrackbar('Area_max','Parameters', 10000, 10000, empty)


def getContours(img, imgContour):
    
    contours, hierarchy = cv2.findContours(img_canny,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE) #contours is not an image, is a chain of pixel locations
    
    for cnt in contours:
        area = cv2.contourArea(cnt)
        areamin = cv2.getTrackbarPos('Area_min', 'Parameters')
        areamax = cv2.getTrackbarPos('Area_max', 'Parameters')
        if area > areamin and area < areamax:
            cv2.drawContours(imgContour, cnt,-1,(255, 0, 255),2)
            perimeter = cv2.arcLength(cnt, True)      # perimeter of contour c (curved length)
            epsilon = 0.02*perimeter    # parameter of polygon approximation: smaller values provide more vertices
            approx = cv2.approxPolyDP(cnt, epsilon, True)
            x,y,w,h = cv2.boundingRect(approx)
            cv2.rectangle(imgContour, (x,y), (x+w,y+h), (255, 0, 0), 2)
#             cv2.putText(imgContour, 'Points:' + str(len(approx)), (x + w + 20, y + 20), cv2.FONT_HERSHEY_COMPLEX, .7, (0, 255,0),2)
#             cv2.putText(imgContour, 'Areas:' + str(int(area)), (x + w + 20, y + 45), cv2.FONT_HERSHEY_COMPLEX, .7, (0, 255,0),2)


while True:
    success, img = vc.read()
    imgContour = img.copy()
    img_blur = cv2.GaussianBlur(img, (7,7),1)
    img_gray = cv2.cvtColor(img_blur, cv2.COLOR_BGR2GRAY)   # convert to B/W
    
    threshold1 = cv2.getTrackbarPos('Threshold1', 'Parameters')
    threshold2 = cv2.getTrackbarPos('Threshold2', 'Parameters')
    img_canny = cv2.Canny(img_gray, threshold1, threshold2)    # standard canny edge detector
    kernel = np.ones((3, 3))
    img_dil = cv2.dilate(img_canny, kernel, iterations=1)
    
    retval = cv2.MatchShapes(contours[0], classifier , CV_CONTOURS_MATCH_I1, 0)
    print(retval)
    
    getContours(img_dil,imgContour)
       
    cv2.imshow('Result', img_canny)
    cv2.imshow('Result2', imgContour)


    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cv2.destroyAllWindows()
vc.release()        


FileNotFoundError: [Errno 2] No such file or directory: 'Uno_Classifier.sav'